##### class inheritence 

In [13]:
class Person:
    description = "general person"
    
    def __init__(self, name, age):
        self.name = name
        self.age = age
    
    def speak(self):
        print(f"My name is {self.name} and I am {self.age} year old.")
        
    def eat(self, food):
        print(f"{self.name} likes to eat {food}")
    
    def action(self):
        print(f"{self.name} jumps")
    

In [20]:
class Baby(Person):
    description = "new baby"
    
    def speak(self):
        print("ba ba ba ba ba")
    
    def nap(self):
        print(f"{self.name} like to have a nap")

In [21]:
person = Person("Mike", 30)
baby = Baby("Ian", 1)
baby.speak()
baby.eat("baby food")
baby.action()
print("-----------")
print(Person.description)
print(Baby.description)
print("-----------")
print(isinstance(baby, Person))
print(isinstance(baby, object))

ba ba ba ba ba
Ian likes to eat baby food
Ian jumps
-----------
general person
new baby
-----------
True
True


In [30]:
def a(length, **kwargs):
    print("len:", length)
    print([len(kwargs), type(kwargs)])
    for k in kwargs:
        print("{}: {}".format(k, kwargs[k]))

In [34]:
a(length=1,base=2)

len: 1
[1, <class 'dict'>]
base: 2


In [51]:
class sth:
    def __init__(self, **kwargs):
        super().__init__(**kwargs)
        pass

In [67]:
class Rectangle:
    def __init__(self, length, width, **kwargs):
        self.length = length
        self.width = width
        print("Rectangle",kwargs)
#         del kwargs["base"]
        print("Rectangle2",kwargs)
        super().__init__(**kwargs)

class Square(Rectangle):
    def __init__(self, length, **kwargs):
        print("Square",kwargs)
        super().__init__(length=length, width=length, **kwargs)

class Triangle:
    def __init__(self, base, height, **kwargs):
        self.base = base
        self.height = height
        print("Triangle",kwargs)
        super().__init__(**kwargs)

class RightPyramid(Triangle, Square):
    def __init__(self, base, slant_height, **kwargs):
        self.base = base
        self.slant_height = slant_height
        kwargs['height'] = slant_height
        kwargs['length'] = base
        print("RightPyramid",kwargs)
        super().__init__(base=base, **kwargs)

In [68]:
a = RightPyramid(12, 34)

RightPyramid {'height': 34, 'length': 12}
Triangle {'length': 12}
Square {}
Rectangle {}
Rectangle2 {}


###### immutable data structure

In [112]:
import os
import itertools
import collections
import multiprocessing
import concurrent.futures
from functools import reduce
from pprint import pprint

In [75]:
def is_prime(num):
    if num > 1: 
        for i in range(2, num//2): 
            # If num is divisible by any number between  
            # 2 and n / 2, it is not prime  
            if (num % i) == 0: 
                return False
    else:
        return False
    return True

In [79]:
print(list(filter(lambda x: is_prime(x), range(100))))

[2, 3, 4, 5, 7, 11, 13, 17, 19, 23, 29, 31, 37, 41, 43, 47, 53, 59, 61, 67, 71, 73, 79, 83, 89, 97]


In [96]:
# we can change the example to company's funding status, active, industry, country/state, vc_backed
# age: can become 'year_founded', company can be CES participates
Scientist = collections.namedtuple('Scientist', [
    'name',
    'field',
    'born',
    'nobel',
])

scientists = (
    Scientist(name='Ada Lovelace', field='math', born=1815, nobel=False),
    Scientist(name='Emmy Noether', field='math', born=1882, nobel=False),
    Scientist(name='Marie Curie', field='math', born=1867, nobel=True),
    Scientist(name='Tu Youyou', field='physics', born=1930, nobel=True),
    Scientist(name='Ada Yonath', field='chemistry', born=1939, nobel=True),
    Scientist(name='Vera Rubin', field='astronomy', born=1928, nobel=False),
    Scientist(name='Sally Ride', field='physics', born=1951, nobel=False),
)

In [97]:
pprint(tuple(filter(lambda x: x.nobel is True, scientists)))

(Scientist(name='Marie Curie', field='math', born=1867, nobel=True),
 Scientist(name='Tu Youyou', field='physics', born=1930, nobel=True),
 Scientist(name='Ada Yonath', field='chemistry', born=1939, nobel=True))


In [98]:
pprint(tuple(filter(lambda x: x.nobel and x.field == "physics", scientists)))

(Scientist(name='Tu Youyou', field='physics', born=1930, nobel=True),)


In [84]:
names_and_ages = tuple(map(lambda x: {'name': x.name, 'age': 2020 - x.born}, scientists))

In [86]:
pprint(names_and_ages)

({'age': 205, 'name': 'Ada Lovelace'},
 {'age': 138, 'name': 'Emmy Noether'},
 {'age': 153, 'name': 'Marie Curie'},
 {'age': 90, 'name': 'Tu Youyou'},
 {'age': 81, 'name': 'Ada Yonath'},
 {'age': 92, 'name': 'Vera Rubin'},
 {'age': 69, 'name': 'Sally Ride'})


In [99]:
tuple({'name': x.name, 'age': 2020 - x.born} for x in scientists)

({'name': 'Ada Lovelace', 'age': 205},
 {'name': 'Emmy Noether', 'age': 138},
 {'name': 'Marie Curie', 'age': 153},
 {'name': 'Tu Youyou', 'age': 90},
 {'name': 'Ada Yonath', 'age': 81},
 {'name': 'Vera Rubin', 'age': 92},
 {'name': 'Sally Ride', 'age': 69})

In [92]:
total_age = reduce(lambda acc, val: acc + val['age'], names_and_ages, 0)
print(total_age)

828


In [103]:
def reducer(acc, val):
    acc[val.field].append(val.name)
    return acc

def reducer2(acc, val):
    acc.setdefault(val.field, []).append(val.name)
    return acc

In [100]:
scientists_by_field = reduce(reducer, scientists,
                             {'math': [], 'physics': [], 'chemistry': [], 'astronomy': []}
                            )
pprint(scientists_by_field)

{'astronomy': ['Vera Rubin'],
 'chemistry': ['Ada Yonath'],
 'math': ['Ada Lovelace', 'Emmy Noether', 'Marie Curie'],
 'physics': ['Tu Youyou', 'Sally Ride']}


In [104]:
scientists_by_field2 = reduce(reducer2, scientists, {})
pprint(scientists_by_field2)

{'astronomy': ['Vera Rubin'],
 'chemistry': ['Ada Yonath'],
 'math': ['Ada Lovelace', 'Emmy Noether', 'Marie Curie'],
 'physics': ['Tu Youyou', 'Sally Ride']}


In [105]:
scientists_by_field3 = reduce(reducer, scientists, collections.defaultdict(list))
pprint(scientists_by_field3)

defaultdict(<class 'list'>,
            {'astronomy': ['Vera Rubin'],
             'chemistry': ['Ada Yonath'],
             'math': ['Ada Lovelace', 'Emmy Noether', 'Marie Curie'],
             'physics': ['Tu Youyou', 'Sally Ride']})


In [107]:
scientists_by_field4 = {item[0]: list(item[1])
                        for item in itertools.groupby(scientists, lambda x: x.field)}
pprint(scientists_by_field4)

{'astronomy': [Scientist(name='Vera Rubin', field='astronomy', born=1928, nobel=False)],
 'chemistry': [Scientist(name='Ada Yonath', field='chemistry', born=1939, nobel=True)],
 'math': [Scientist(name='Ada Lovelace', field='math', born=1815, nobel=False),
          Scientist(name='Emmy Noether', field='math', born=1882, nobel=False),
          Scientist(name='Marie Curie', field='math', born=1867, nobel=True)],
 'physics': [Scientist(name='Sally Ride', field='physics', born=1951, nobel=False)]}


In [113]:
# can change to use timer decorator
def transform(x):
    print(f'Processing {os.getpid()}record {x.name}')
    result = {'name': x.name, 'age': 2020 - x.born}
    print(f'{os.getpid()} Done processing record {x.name}')
    return result

In [114]:

pprint(tuple(map(transform, scientists)))

Processing 30022record Ada Lovelace
30022 Done processing record Ada Lovelace
Processing 30022record Emmy Noether
30022 Done processing record Emmy Noether
Processing 30022record Marie Curie
30022 Done processing record Marie Curie
Processing 30022record Tu Youyou
30022 Done processing record Tu Youyou
Processing 30022record Ada Yonath
30022 Done processing record Ada Yonath
Processing 30022record Vera Rubin
30022 Done processing record Vera Rubin
Processing 30022record Sally Ride
30022 Done processing record Sally Ride
({'age': 205, 'name': 'Ada Lovelace'},
 {'age': 138, 'name': 'Emmy Noether'},
 {'age': 153, 'name': 'Marie Curie'},
 {'age': 90, 'name': 'Tu Youyou'},
 {'age': 81, 'name': 'Ada Yonath'},
 {'age': 92, 'name': 'Vera Rubin'},
 {'age': 69, 'name': 'Sally Ride'})


In [116]:
pool = multiprocessing.Pool(processes=len(scientists))
result = pool.map(transform, scientists)

Processing 38750record Emmy Noether
Processing 38754record Vera Rubin
Processing 38753record Ada Yonath
Processing 38749record Ada Lovelace
Processing 38751record Marie Curie
Processing 38752record Tu Youyou
Processing 38755record Sally Ride
38749 Done processing record Ada Lovelace
38753 Done processing record Ada Yonath
38750 Done processing record Emmy Noether
38754 Done processing record Vera Rubin
38751 Done processing record Marie Curie
38752 Done processing record Tu Youyou
38755 Done processing record Sally Ride


In [119]:
with concurrent.futures.ProcessPoolExecutor() as executor:
    result = executor.map(transform, scientists)

Processing 38765record Ada Lovelace
Processing 38766record Emmy Noether
Processing 38769record Ada Yonath
Processing 38770record Vera Rubin
Processing 38767record Marie Curie
Processing 38771record Sally Ride
Processing 38768record Tu Youyou
38766 Done processing record Emmy Noether
38765 Done processing record Ada Lovelace
38767 Done processing record Marie Curie
38768 Done processing record Tu Youyou
38770 Done processing record Vera Rubin
38769 Done processing record Ada Yonath
38771 Done processing record Sally Ride


In [121]:
import threading
thread_local = threading.local()


def get_session():
    if not hasattr(thread_local, "session"):
        thread_local.session = requests.Session()
    return thread_local.session


def download_site(url):
    session = get_session()
    with session.get(url) as response:
        print(f"Read {len(response.content)} from {url}")


def download_all_sites(sites):
    with concurrent.futures.ThreadPoolExecutor(max_workers=5) as executor:
        executor.map(download_site, sites)

In [123]:
sentence = 'The rocket, who was named Ted, came back from Mars because he missed his friends.'
def is_consonant(letter):
    vowels = 'aeiou'
    return letter.isalpha() and letter.lower() not in vowels
consonants = [i for i in sentence if is_consonant(i)]
print("consonants:", consonants)

consonants: ['T', 'h', 'r', 'c', 'k', 't', 'w', 'h', 'w', 's', 'n', 'm', 'd', 'T', 'd', 'c', 'm', 'b', 'c', 'k', 'f', 'r', 'm', 'M', 'r', 's', 'b', 'c', 's', 'h', 'm', 's', 's', 'd', 'h', 's', 'f', 'r', 'n', 'd', 's']


#### unit test for lambda function

In [131]:
import unittest
import pytest

In [130]:
square = lambda x: x**2

In [133]:
class LambdaTest(unittest.TestCase):
    def test_squared(self):
        self.assertEqual(square(2), 4)
        
    def test_zero(self):
        self.assertEqual(square(0), 0)
        
    def test_negative(self):
        self.assertEqual(square(-2), 4)